In [1]:
import json
from pathlib import Path
_API_DIR = Path("/Users/joregan/riksdag/riksdag-api-out")

In [2]:
def endswith_list(text, items):
    for it in items:
        if text.endswith(it):
            return True
    return False

In [3]:
def viddata_get_single_stream(videodata, hires=True):
    videos = []
    if videodata is None:
        return []
    if 'streams' not in videodata:
        #raise Exception("videodata is missing 'streams'")
        return []
    if videodata['streams'] is None:
        return []
    if 'files' not in videodata['streams']:
        #raise Exception("videodata['streams'] is missing 'files'")
        return []
    if type(videodata['streams']['files']) == list:
        for vfile in videodata['streams']['files']:
            for bw in vfile['bandwidth']:
                if hires and bw['name'] == 'Hög kvalitet':
                    videos.append(bw['downloadurl'])
                elif not hires and bw['name'] == 'Låg kvalitet':
                    videos.append(bw['downloadurl'])
    else:
        #raise Exception(f"Expected a list, got {type(videodata['streams']['files'])}")
        return []
    return videos

def viddata_get_streams(videodata, hires=True):
    output = []
    if 'videodata' not in videodata:
        #raise Exception("'videodata' missing")
        return []
    for vdata in videodata['videodata']:
        output += viddata_get_single_stream(vdata, hires)
    return output

def viddata_from_file(videofile, hires=True):
    with open(videofile) as jsonf:
        data = json.load(jsonf)
        return viddata_get_streams(data, hires)

In [4]:
def json_matches_years(filename, years):
    ret_val = False
    with open(filename) as f:
        data = json.load(f)
        if not "videodata" in data:
            #raise Exception(f"File {filename} missing key 'videodata'")
            return False
        videodata = data["videodata"]
        if videodata is None:
            print(f"Empty videodata: {filename}")
            return False
        for vdata in videodata:
            if vdata is None:
                print(f"Empty videodata: {filename}")
                return False
            if "debatedate" in vdata and vdata["debatedate"] is not None and vdata["debatedate"] != "":
                date = vdata["debatedate"]
                if endswith_list(date.strip(), years):
                    return True
    return False

In [35]:
matches = []
for file in _API_DIR.glob("H*"):
    if json_matches_years(file, ["2017", "2018"]):
        matches.append(str(file))

Empty videodata: /Users/joregan/riksdag/riksdag-api-out/H8C120210621zz


In [37]:
with open("2017-2018.txt", "w") as outf:
    for m in matches:
        outf.write(m + "\n")

In [7]:
with open("2017-2018-videos.txt", "w") as outf:
    for file in _API_DIR.glob("H*"):
        if json_matches_years(file, ["2017", "2018"]):
            videos = viddata_from_file(file)
            vidsout = "\t".join(videos)
            outf.write(f"{file.stem}\t{vidsout}\n")

Empty videodata: /Users/joregan/riksdag/riksdag-api-out/H8C120210621zz


In [8]:
def get_speaker_data(data):
    output = []
    if not "videodata" in data:
        #raise Exception(f"File {filename} missing key 'videodata'")
        return []
    for vdata in data["videodata"]:
        if "speakers" in vdata:
            for speaker in vdata["speakers"]:
                output.append(speaker)
    return output

In [9]:
with open("/Users/joregan/riksdag/riksdag-api-out/H501CU20") as inp:
    vdata = json.load(inp)
    speakers = get_speaker_data(vdata)
speakers

[{'start': 25,
  'duration': 271,
  'text': 'Caroline Szyber (KD)',
  'party': 'KD',
  'subid': '1100180502423323222',
  'active': False,
  'committee': None,
  'description': None,
  'number': '1',
  'anftext': '<p>Fru talman! Synen på konsumtion och krediter har förändrats under åren, och i dag lånar svenskarna alltmer. Att hamna i skuldfällan och i värsta fall överskuldsättning är förödande för den enskilde och kostar även samhället stora pengar. Svenska hushåll är högt skuldsatta, såväl i ett historiskt som i ett internationellt perspektiv. Målsättningen bör vara en fungerande marknad där man både skapar en återhållsamhet och en tillgänglig lånemarknad.</p><p>Livslång skuldsättning kan drabba vem som helst. Kreditsamhället är under ständig utveckling. Krediter ökar möjligheterna för hushållen att planera sin ekonomi men riskerar också att bidra till överskuldsättning. Synen på konsumtion och krediter har förändrats under åren, och i dag lånar som sagt svenskarna alltmer.</p><p>Livs